# COMP433 - Blood Smear Analysis Object Detection
### This document demonstrates how to perform object detection on blood cell images using the YOLOv11 model and the BCCD dataset form RoboFlow. It covers setting up the environment, downloading the dataset, training the model, and evaluating the results.

In [ ]:
# Check the available GPU
!nvidia-smi

Mon Nov 10 17:22:23 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   58C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
# Get the current working directory
from google.colab import drive
drive.mount('/content/drive')

HOME = "/content/drive/MyDrive/Colab_Notebooks/COMP433/Project" # Path to where the Project folder is stored in your drive. Modify if needed

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls $HOME

'COMP433 - Final Report.gdoc'
'COMP433 - Project Progress Report.gdoc'
 COMP433-SOTA-Project-Group_17.ipynb
'Copy of COMP433-SOTA-Project-Group_17.ipynb'
'Copy of Copy of COMP433-SOTA-Project-Group_17.ipynb'
 datasets
 gen_image_test_1.jpg
 gen_image_test_3.jpg
 gen_image_test_4.jpg
 runs
 yolo11_experiment_results.csv


By default Google CoLab already has PyTorch installed, this is just showing that the depoendenceies are already installed

In [ ]:
# List installed torch packages to confirm installation
!pip list | grep torch

torch                                    2.8.0+cu126
torchao                                  0.10.0
torchaudio                               2.8.0+cu126
torchdata                                0.11.0
torchsummary                             1.5.1
torchtune                                0.6.1
torchvision                              0.23.0+cu126


In [ ]:
# Install ultralytics and roboflow libraries
%pip install ultralytics roboflow

# Import ultralytics and run checks to verify the environment
import ultralytics
ultralytics.checks()

Ultralytics 8.3.227 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 39.5/112.6 GB disk)


In [ ]:
# Create a directory for datasets and change the current directory to it
!mkdir {HOME}/datasets

In [ ]:
%cd {HOME}/datasets

# Retrieve Roboflow API key from Colab userdata and initialize Roboflow
from google.colab import userdata
from roboflow import Roboflow

ROBOFLOW_API_KEY = userdata.get('ROBOFLOW_API_KEY')
rf = Roboflow(api_key=ROBOFLOW_API_KEY)

# Download the BCCD dataset from Roboflow in yolov11 format
# workspace("joseph-nelson").project("bccd")
# workspace("personalworkspace-ttvxm").project("wbc---smear-p8aip")
# workspace("davide-1zkpi").project("blood-fzrmt")
# workspace("telkom-wfznr").project("blood-chmma")
# workspace("trombosit").project("platelet-gvbae")
# workspace("martijn-kok-5kyxb").project("blood-platelets")

/content/drive/MyDrive/Colab_Notebooks/COMP433/Project/datasets


In [ ]:
project = rf.workspace("personalworkspace-ttvxm").project("blood-smear-components-yolov11-f6mus")
version = project.version(2)
dataset = version.download("yolov11")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Blood-Smear-Components---yolov11-2 in yolov11:: 100%|██████████| 13216/13216 [01:50<00:00, 119.92it/s]


- Training 3 version of yolov11 n (nano), s (small), m (medium)
- epochs: 450
- patience: 100
- batch: 8, 16, 32, 64
- optimizer: auto
- weight_decay	default	0.0005


In [ ]:
from ultralytics import YOLO
import pandas as pd
import torch
import os

# ====== SETUP ======
DRIVE_BASE = HOME  # e.g. "/content/drive/MyDrive"
DATA_PATH = f"{DRIVE_BASE}/datasets/Blood-Smear-Components---yolov11-2/data.yaml"
RESULTS_CSV = os.path.join(DRIVE_BASE, "yolo11_experiment_results.csv")
RUNS_DIR = os.path.join(DRIVE_BASE, "runs")

os.makedirs(RUNS_DIR, exist_ok=True)

# ====== CONFIGURATION ======
model_names = ["yolo11n.pt", "yolo11m.pt"]
epochs = 450
batches = [8, 16, 32, 64]
optimizer = "auto"
patience = 100

# ====== LOAD EXISTING RESULTS IF ANY ======
if os.path.exists(RESULTS_CSV):
    print(f"📂 Found existing results file at {RESULTS_CSV}, resuming from it...")
    if os.path.getsize(RESULTS_CSV) > 0:
        try:
            results_table = pd.read_csv(RESULTS_CSV).to_dict("records")
            completed_experiments = {r["experiment"] for r in results_table}
        except pd.errors.EmptyDataError:
            print("⚠️ Results file malformed, starting fresh.")
            results_table = []
            completed_experiments = set()
    else:
        print("⚠️ Results file is empty, starting fresh.")
        results_table = []
        completed_experiments = set()
else:
    results_table = []
    completed_experiments = set()

# ====== MAIN TRAINING LOOP ======
for model_name in model_names:
    for batch in batches:
        exp_name = f"{model_name.split('.')[0]}_b{batch}"
        print(f"\n🚀 Starting experiment: {exp_name}")

        # Skip already completed experiments
        if exp_name in completed_experiments:
            print(f"⏩ Skipping {exp_name} (already completed)")
            continue

        # Path to expected checkpoint (Ultralytics structure)
        EXP_DIR = os.path.join(RUNS_DIR, exp_name)
        WEIGHTS_DIR = os.path.join(EXP_DIR, "weights")
        MODEL_CHECKPOINT = os.path.join(WEIGHTS_DIR, "last.pt")

        # ====== INITIALIZE MODEL ======
        if os.path.exists(MODEL_CHECKPOINT):
            print(f"🔄 Found checkpoint for {exp_name}, resuming training...")
            model = YOLO(MODEL_CHECKPOINT)
            resume_flag = True
        else:
            print(f"🚀 Starting new training for {exp_name}...")
            model = YOLO(model_name)
            resume_flag = False

        try:
            # ====== TRAIN ======
            results = model.train(
                data=DATA_PATH,
                epochs=epochs,
                batch=batch,
                patience=patience,
                optimizer=optimizer,
                name=exp_name,                  # experiment folder name
                project=RUNS_DIR,               # stored in /runs/
                exist_ok=True,
                resume=resume_flag,
                save=True,
                save_period=1
            )

            # ====== VALIDATE ======
            print(f"📊 Validating {exp_name} ...")
            val_results = model.val()

            # ====== SAVE RESULTS ======
            results_table.append({
                "experiment": exp_name,
                "model": model_name,
                "epochs": epochs,
                "batch": batch,
                "optimizer": optimizer,
                "mAP50": val_results.box.map50,
                "mAP50-95": val_results.box.map
            })

            pd.DataFrame(results_table).to_csv(RESULTS_CSV, index=False)
            print(f"💾 Saved results to {RESULTS_CSV}")

            # Free GPU memory
            torch.cuda.empty_cache()

        except Exception as e:
            print(f"⚠️ Error during {exp_name}: {e}")
            pd.DataFrame(results_table).to_csv(RESULTS_CSV, index=False)
            print("💾 Progress saved before crash.")
            break

print("\n✅ All experiments complete! Final results saved to:")
print(RESULTS_CSV)

📂 Found existing results file at /content/drive/MyDrive/Colab_Notebooks/COMP433/Project/yolo11_experiment_results.csv, resuming from it...

🚀 Starting experiment: yolo11n_b8
⏩ Skipping yolo11n_b8 (already completed)

🚀 Starting experiment: yolo11n_b16
🔄 Found checkpoint for yolo11n_b16, resuming training...
Ultralytics 8.3.227 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/Colab_Notebooks/COMP433/Project/datasets/Blood-Smear-Components---yolov11-2/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=450, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=Fal